# Protein MD Setup tutorial using BioExcel Building Blocks (biobb)
**Based on the official GROMACS tutorial:** [http://www.mdtutorials.com/gmx/lysozyme/index.html](http://www.mdtutorials.com/gmx/lysozyme/index.html)
***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **Lysozyme** protein (PDB code 1AKI). 
***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_gromacs](https://github.com/bioexcel/biobb_gromacs): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_md_setup.git
cd biobb_wf_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_GMX_MDsetup_tutorial
jupyter nbextension enable python-markdown/main
jupyter-notebook biobb_wf_md_setup/notebooks/biobb_MDsetup_tutorial.ipynb
```

Please execute the following commands before launching the Jupyter Notebook if you experience some issues with widgets such as NGL View (3D molecular visualization):

```console
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
```

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Fix Protein Structure](#fix)
 4. [Create Protein System Topology](#top)
 5. [Create Solvent Box](#box)
 6. [Fill the Box with Water Molecules](#water)
 7. [Adding Ions](#ions)
 8. [Energetically Minimize the System](#min)
 9. [Equilibrate the System (NVT)](#nvt)
 10. [Equilibrate the System (NPT)](#npt)
 11. [Free Molecular Dynamics Simulation](#free)
 12. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 13. [Output Files](#output)
 14. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [18]:
import nglview
import ipywidgets

pdbCode = "1AKI"

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [19]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'
prop = {
    'pdb_code': pdbCode
}

#Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2021-06-23 07:31:28,686 [MainThread  ] [INFO ]  Downloading: 1aki from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb1aki.ent
2021-06-23 07:31:28,873 [MainThread  ] [INFO ]  Writting pdb to: 1AKI.pdb
2021-06-23 07:31:28,875 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


/anaconda3/envs/biobb_dev_37/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**:    

In [20]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<img src='ngl1.png'></img>

<a id="fix"></a>
***
## Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.

***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [21]:
# Check & Fix PDB
# Import module
from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = pdbCode + '_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=downloaded_pdb, 
             output_pdb_path=fixed_pdb)

2021-06-23 07:31:29,579 [MainThread  ] [INFO ]  check_structure -i 1AKI.pdb -o 1AKI_fixed.pdb --force_save fixside --fix ALL

2021-06-23 07:31:29,585 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:31:29,589 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.7.2                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure 1AKI.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution: None A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Final Num. models: 1
Final Num. chains: 1 (A: Protein)
Final Num. residues:  1

0

### Visualizing 3D structure
Visualizing the fixed **PDB structure** using **NGL**. In this particular example, the checking step didn't find any issue to be solved, so there is no difference between the original structure and the fixed one.   

In [22]:
# Show protein
view = nglview.show_structure_file(fixed_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl2.png'></img>

<a id="top"></a>
***
## Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_gromacs.gromacs.pdb2gmx**
***

In [23]:
# Create system topology
# Import module
from biobb_gromacs.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro = pdbCode+'_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbCode+'_pdb2gmx_top.zip'

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb, 
        output_gro_path=output_pdb2gmx_gro, 
        output_top_zip_path=output_pdb2gmx_top_zip)

2021-06-23 07:31:29,835 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20190 version detected
2021-06-23 07:31:29,842 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:31:30,419 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f 1AKI_fixed.pdb -o 1AKI_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2021-06-23 07:31:30,421 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:31:30,421 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b
going to rename amber99sb-ildn.ff/dna.r2b
going to rename amber99sb-ildn.ff/rna.r2b
Reading 1AKI_fixed.pdb...
Read '', 1001 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 129 residues with 1001 atoms

  chain  #res #atoms
  1 'A'   129   1001  

Reading residue database... (Amber99sb-ildn)
Processing chain 1 'A' (1001 atoms, 129 residues)
Ide

0

### Visualizing 3D structure
Visualizing the generated **GRO structure** using **NGL**. Note that **hydrogen atoms** were added to the structure by the **pdb2gmx GROMACS tool** when generating the **topology**.    

In [24]:
# Show protein
view = nglview.show_structure_file(output_pdb2gmx_gro)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl3.png'></img>

<a id="box"></a>
***
## Create solvent box
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **cubic box** is used to define the unit cell, with a **distance from the protein to the box edge of 1.0 nm**. The protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_gromacs.gromacs.editconf** 
***

In [25]:
# Editconf: Create solvent box
# Import module
from biobb_gromacs.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = pdbCode+'_editconf.gro'

prop = {
    'box_type': 'cubic',
    'distance_to_molecule': 1.0
}

#Create and launch bb
editconf(input_gro_path=output_pdb2gmx_gro, 
         output_gro_path=output_editconf_gro,
         properties=prop)

2021-06-23 07:31:30,618 [MainThread  ] [INFO ]  GROMACS Editconf 20190 version detected
2021-06-23 07:31:30,619 [MainThread  ] [INFO ]  Centering molecule in the box.
2021-06-23 07:31:30,620 [MainThread  ] [INFO ]  Distance of the box to molecule:   1.00
2021-06-23 07:31:30,621 [MainThread  ] [INFO ]  Box type: cubic
2021-06-23 07:31:30,622 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:31:30,650 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f 1AKI_pdb2gmx.gro -o 1AKI_editconf.gro -d 1.0 -bt cubic -c

2021-06-23 07:31:30,652 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:31:30,653 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 1960 atoms
Volume: 55.8378 nm^3, corresponds to roughly 25100 electrons
No velocities found
    system size :  3.817  4.234  3.454 (nm)
    diameter    :  5.010               (nm)
    center      :  2.781  2.488  0.017 (nm)
    box vectors :  3.817  4.235 

0

<a id="water"></a>
***
## Fill the box with water molecules
Fill the unit cell for the **protein structure system** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_gromacs.gromacs.solvate** 
***

In [26]:
# Solvate: Fill the box with water molecules
from biobb_gromacs.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro = pdbCode+'_solvate.gro'
output_solvate_top_zip = pdbCode+'_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro, 
        output_gro_path=output_solvate_gro, 
        input_top_zip_path=output_pdb2gmx_top_zip, 
        output_top_zip_path=output_solvate_top_zip)

2021-06-23 07:31:30,707 [MainThread  ] [INFO ]  GROMACS Solvate 20190 version detected
2021-06-23 07:31:30,715 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_pdb2gmx_top.zip
2021-06-23 07:31:30,716 [MainThread  ] [INFO ]  to:
2021-06-23 07:31:30,717 [MainThread  ] [INFO ]  ['8917f853-a42b-466d-a186-a0ac33dce06d/p2g.top', '8917f853-a42b-466d-a186-a0ac33dce06d/posre.itp']
2021-06-23 07:31:30,718 [MainThread  ] [INFO ]  Unzipping: 
2021-06-23 07:31:30,718 [MainThread  ] [INFO ]  1AKI_pdb2gmx_top.zip
2021-06-23 07:31:30,721 [MainThread  ] [INFO ]  To: 
2021-06-23 07:31:30,722 [MainThread  ] [INFO ]  8917f853-a42b-466d-a186-a0ac33dce06d/p2g.top
2021-06-23 07:31:30,723 [MainThread  ] [INFO ]  8917f853-a42b-466d-a186-a0ac33dce06d/posre.itp
2021-06-23 07:31:30,727 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:31:31,271 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright solvate -cp 1AKI_editcon

0

### Visualizing 3D structure
Visualizing the **protein system** with the newly added **solvent box** using **NGL**.<br> Note the **cubic box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the cube.

In [27]:
# Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl4.png'></img>

<a id="ions"></a>
***
## Adding ions
Add ions to neutralize the **protein structure** charge
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_gromacs.gromacs.genion** 
***

<a id="ionsStep1"></a>
### Step 1: Creating portable binary run file for ion generation
A simple **energy minimization** molecular dynamics parameters (mdp) properties will be used to generate the portable binary run file for **ion generation**, although **any legitimate combination of parameters** could be used in this step.

In [28]:
# Grompp: Creating portable binary run file for ion generation
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppion_tpr = pdbCode+'_gppion.tpr'
prop = {
    'simulation_type': 'minimization'
}

# Create and launch bb
grompp(input_gro_path=output_solvate_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,  
       properties=prop)

2021-06-23 07:31:31,562 [MainThread  ] [INFO ]  GROMACS Grompp 20190 version detected
2021-06-23 07:31:31,567 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_solvate_top.zip
2021-06-23 07:31:31,568 [MainThread  ] [INFO ]  to:
2021-06-23 07:31:31,569 [MainThread  ] [INFO ]  ['59cdbc80-d19c-4132-8627-fb4653bb1bd2/p2g.top', '59cdbc80-d19c-4132-8627-fb4653bb1bd2/posre.itp']
2021-06-23 07:31:31,570 [MainThread  ] [INFO ]  Unzipping: 
2021-06-23 07:31:31,571 [MainThread  ] [INFO ]  1AKI_solvate_top.zip
2021-06-23 07:31:31,572 [MainThread  ] [INFO ]  To: 
2021-06-23 07:31:31,572 [MainThread  ] [INFO ]  59cdbc80-d19c-4132-8627-fb4653bb1bd2/p2g.top
2021-06-23 07:31:31,573 [MainThread  ] [INFO ]  59cdbc80-d19c-4132-8627-fb4653bb1bd2/posre.itp
2021-06-23 07:31:31,575 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:31:31,971 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f 2a0bea2e-dec5-4

0

<a id="ionsStep2"></a>
### Step 2: Adding ions to neutralize the system
Replace **solvent molecules** with **ions** to **neutralize** the system.

In [29]:
# Genion: Adding ions to neutralize the system
from biobb_gromacs.gromacs.genion import genion

# Create prop dict and inputs/outputs
output_genion_gro = pdbCode+'_genion.gro'
output_genion_top_zip = pdbCode+'_genion_top.zip'
prop={
    'neutral':True
}

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr, 
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_top_zip_path=output_genion_top_zip, 
       properties=prop)

2021-06-23 07:31:32,026 [MainThread  ] [INFO ]  GROMACS Genion 20190 version detected
2021-06-23 07:31:32,031 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_solvate_top.zip
2021-06-23 07:31:32,031 [MainThread  ] [INFO ]  to:
2021-06-23 07:31:32,032 [MainThread  ] [INFO ]  ['477df766-3dcc-44ba-abb8-cfaf66894e66/p2g.top', '477df766-3dcc-44ba-abb8-cfaf66894e66/posre.itp']
2021-06-23 07:31:32,032 [MainThread  ] [INFO ]  Unzipping: 
2021-06-23 07:31:32,033 [MainThread  ] [INFO ]  1AKI_solvate_top.zip
2021-06-23 07:31:32,033 [MainThread  ] [INFO ]  To: 
2021-06-23 07:31:32,034 [MainThread  ] [INFO ]  477df766-3dcc-44ba-abb8-cfaf66894e66/p2g.top
2021-06-23 07:31:32,034 [MainThread  ] [INFO ]  477df766-3dcc-44ba-abb8-cfaf66894e66/posre.itp
2021-06-23 07:31:32,035 [MainThread  ] [INFO ]  To reach up 0.05 mol/litre concentration
2021-06-23 07:31:32,035 [MainThread  ] [INFO ]  Not using any container
2021-06-23 

0

### Visualizing 3D structure
Visualizing the **neutralized protein system** with the newly added **ions** using **NGL**

In [30]:
# Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl5.png'></img>

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **protein system** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **system** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
The **minimization** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **energy minimization**:

-  integrator  = steep ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01 ; Minimization step size (nm)
-  nsteps      = 50000 ; Maximum number of (minimization) steps to perform

In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, but the **maximum force** is placed at **500 KJ/mol\*nm^2**, and the **maximum number of steps** to perform (if the maximum force is not reached) to **5,000 steps**. 

In [31]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppmin_tpr = pdbCode+'_gppmin.tpr'
prop = {
    'mdp':{
        'emtol':'500',
        'nsteps':'5000'
    },
    'simulation_type': 'minimization'
}

# Create and launch bb
grompp(input_gro_path=output_genion_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppmin_tpr,  
       properties=prop)

2021-06-23 07:31:32,521 [MainThread  ] [INFO ]  GROMACS Grompp 20190 version detected
2021-06-23 07:31:32,526 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_genion_top.zip
2021-06-23 07:31:32,527 [MainThread  ] [INFO ]  to:
2021-06-23 07:31:32,530 [MainThread  ] [INFO ]  ['18aff72e-a338-4ddd-ace1-77c076b59652/p2g.top', '18aff72e-a338-4ddd-ace1-77c076b59652/posre.itp']
2021-06-23 07:31:32,531 [MainThread  ] [INFO ]  Unzipping: 
2021-06-23 07:31:32,532 [MainThread  ] [INFO ]  1AKI_genion_top.zip
2021-06-23 07:31:32,533 [MainThread  ] [INFO ]  To: 
2021-06-23 07:31:32,533 [MainThread  ] [INFO ]  18aff72e-a338-4ddd-ace1-77c076b59652/p2g.top
2021-06-23 07:31:32,534 [MainThread  ] [INFO ]  18aff72e-a338-4ddd-ace1-77c076b59652/posre.itp
2021-06-23 07:31:32,535 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:31:32,954 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f 9abe032d-e6e4-4f8

0

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step. 

In [32]:
# Mdrun: Running minimization
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = pdbCode+'_min.trr'
output_min_gro = pdbCode+'_min.gro'
output_min_edr = pdbCode+'_min.edr'
output_min_log = pdbCode+'_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr, 
      output_trr_path=output_min_trr, 
      output_gro_path=output_min_gro, 
      output_edr_path=output_min_edr, 
      output_log_path=output_min_log)

2021-06-23 07:31:32,993 [MainThread  ] [INFO ]  GROMACS Mdrun 20190 version detected
2021-06-23 07:31:32,994 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:33:47,475 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 1AKI_gppmin.tpr -o 1AKI_min.trr -c 1AKI_min.gro -e 1AKI_min.edr -g 1AKI_min.log

2021-06-23 07:33:47,476 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:33:47,477 [MainThread  ] [INFO ]                         :-) GROMACS - gmx mdrun, 2019 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 1AKI_gppmin.tpr -o 1AKI_min.trr -c 1AKI_min.gro -e 1AKI_min.edr -g 1AKI_min.log

Reading file 1AKI_gppmin.tpr, VERSION 2019 (single precision)
Using 1 MPI thread
Using 8 OpenMP threads 


Steepest Descents:
   Tolerance (Fmax)   =  5.00000e+02
   Number of steps    =         500

0

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [33]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = pdbCode+'_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=prop)

2021-06-23 07:33:47,502 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:33:47,532 [MainThread  ] [INFO ]  gmx energy -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_min.edr -o 1AKI_min_ene.xvg -xvg none < 33c49c25-632e-4fde-bc40-6e482ff2ce92/instructions.in

2021-06-23 07:33:47,534 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:33:47,534 [MainThread  ] [INFO ]  
Statistics over 2493 steps [ 0.0000 through 2492.0000 ps ], 1 data sets
All statistics are over 1973 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -585877       9000    22960.4   -58712.9  (kJ/mol)

2021-06-23 07:33:47,537 [MainThread  ] [INFO ]                         :-) GROMACS - gmx energy, 2019 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     He

0

In [34]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
}

plotly.offline.iplot(fig)

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein system** in **NVT ensemble** (constant Number of particles, Volume and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNVTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNVTStep2): Equilibrate the **protein system** with **NVT** ensemble.
- [Step 3](#eqNVTStep3): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process. 
***
**Building Blocks** used:
- [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
- [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
- [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNVTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NVT)
The **nvt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NVT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl                   = no
-  gen_vel                  = yes
-  gen_temp                 = 300
-  gen_seed                 = -1

In this particular example, the default parameters will be used: **md** integrator algorithm, a **step size** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a temperature of **300K**.

*Please note that for the sake of time this tutorial is only running 10ps of NVT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/06_equil.html) the simulated time was 100ps.*

In [35]:
# Grompp: Creating portable binary run file for NVT Equilibration
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnvt_tpr = pdbCode+'_gppnvt.tpr'
prop = {
    'mdp':{
        'nsteps': 5000,
        'dt': 0.002,
        'Define': '-DPOSRES',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    },
    'simulation_type': 'nvt'
}

# Create and launch bb
grompp(input_gro_path=output_min_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppnvt_tpr,  
       properties=prop)

2021-06-23 07:33:48,104 [MainThread  ] [INFO ]  GROMACS Grompp 20190 version detected
2021-06-23 07:33:48,110 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_genion_top.zip
2021-06-23 07:33:48,111 [MainThread  ] [INFO ]  to:
2021-06-23 07:33:48,111 [MainThread  ] [INFO ]  ['ead8a156-a898-4418-a017-12c760b89917/p2g.top', 'ead8a156-a898-4418-a017-12c760b89917/posre.itp']
2021-06-23 07:33:48,113 [MainThread  ] [INFO ]  Unzipping: 
2021-06-23 07:33:48,113 [MainThread  ] [INFO ]  1AKI_genion_top.zip
2021-06-23 07:33:48,114 [MainThread  ] [INFO ]  To: 
2021-06-23 07:33:48,115 [MainThread  ] [INFO ]  ead8a156-a898-4418-a017-12c760b89917/p2g.top
2021-06-23 07:33:48,115 [MainThread  ] [INFO ]  ead8a156-a898-4418-a017-12c760b89917/posre.itp
2021-06-23 07:33:48,117 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:33:48,573 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f caab13e6-8b9a-48d

0

<a id="eqNVTStep2"></a>
### Step 2: Running NVT equilibration

In [36]:
# Mdrun: Running Equilibration NVT
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_nvt_trr = pdbCode+'_nvt.trr'
output_nvt_gro = pdbCode+'_nvt.gro'
output_nvt_edr = pdbCode+'_nvt.edr'
output_nvt_log = pdbCode+'_nvt.log'
output_nvt_cpt = pdbCode+'_nvt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnvt_tpr, 
      output_trr_path=output_nvt_trr, 
      output_gro_path=output_nvt_gro, 
      output_edr_path=output_nvt_edr, 
      output_log_path=output_nvt_log, 
      output_cpt_path=output_nvt_cpt)

2021-06-23 07:33:48,606 [MainThread  ] [INFO ]  GROMACS Mdrun 20190 version detected
2021-06-23 07:33:48,609 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:34:48,512 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 1AKI_gppnvt.tpr -o 1AKI_nvt.trr -c 1AKI_nvt.gro -e 1AKI_nvt.edr -g 1AKI_nvt.log -cpo 1AKI_nvt.cpt

2021-06-23 07:34:48,513 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:34:48,513 [MainThread  ] [INFO ]                         :-) GROMACS - gmx mdrun, 2019 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 1AKI_gppnvt.tpr -o 1AKI_nvt.trr -c 1AKI_nvt.gro -e 1AKI_nvt.edr -g 1AKI_nvt.log -cpo 1AKI_nvt.cpt

Reading file 1AKI_gppnvt.tpr, VERSION 2019 (single precision)
Changing nstlist from 10 to 50, rlist from 1 to 1.115

Using 1 MPI thread
Using 8 OpenMP threads 

s

0

<a id="eqNVTStep3"></a>
### Step 3: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [37]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_nvt_temp_xvg = pdbCode+'_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_nvt_edr, 
          output_xvg_path=output_nvt_temp_xvg, 
          properties=prop)

2021-06-23 07:34:48,541 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:34:48,563 [MainThread  ] [INFO ]  gmx energy -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_nvt.edr -o 1AKI_nvt_temp.xvg -xvg none < da6bcf69-f62f-45bd-9aae-55a344f88846/instructions.in

2021-06-23 07:34:48,564 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:34:48,565 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 1 data sets
All statistics are over 51 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 297.635          2    9.45548    10.8383  (K)

2021-06-23 07:34:48,566 [MainThread  ] [INFO ]                         :-) GROMACS - gmx energy, 2019 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berends

0

In [38]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg,'r') as temperature_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

The **npt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a Parrinello-Rahman **pressure coupling** algorithm.

*Please note that for the sake of time this tutorial is only running 10ps of NPT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/07_equil2.html) the simulated time was 100ps.*

In [39]:
# Grompp: Creating portable binary run file for NPT System Equilibration
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = pdbCode+'_gppnpt.tpr'
prop = {
    'mdp':{
        'nsteps':'5000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    },
    'simulation_type': 'npt'
}

# Create and launch bb
grompp(input_gro_path=output_nvt_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppnpt_tpr, 
       input_cpt_path=output_nvt_cpt,  
       properties=prop)

2021-06-23 07:34:48,661 [MainThread  ] [INFO ]  GROMACS Grompp 20190 version detected
2021-06-23 07:34:48,672 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_genion_top.zip
2021-06-23 07:34:48,673 [MainThread  ] [INFO ]  to:
2021-06-23 07:34:48,674 [MainThread  ] [INFO ]  ['18c1da8c-3087-4d3b-bce2-dfa4c5f57196/p2g.top', '18c1da8c-3087-4d3b-bce2-dfa4c5f57196/posre.itp']
2021-06-23 07:34:48,676 [MainThread  ] [INFO ]  Unzipping: 
2021-06-23 07:34:48,676 [MainThread  ] [INFO ]  1AKI_genion_top.zip
2021-06-23 07:34:48,677 [MainThread  ] [INFO ]  To: 
2021-06-23 07:34:48,679 [MainThread  ] [INFO ]  18c1da8c-3087-4d3b-bce2-dfa4c5f57196/p2g.top
2021-06-23 07:34:48,680 [MainThread  ] [INFO ]  18c1da8c-3087-4d3b-bce2-dfa4c5f57196/posre.itp
2021-06-23 07:34:48,682 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:34:49,481 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f db51cb9f-c8bb-486

0

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [40]:
# Mdrun: Running NPT System Equilibration
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = pdbCode+'_npt.trr'
output_npt_gro = pdbCode+'_npt.gro'
output_npt_edr = pdbCode+'_npt.edr'
output_npt_log = pdbCode+'_npt.log'
output_npt_cpt = pdbCode+'_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr, 
      output_trr_path=output_npt_trr, 
      output_gro_path=output_npt_gro, 
      output_edr_path=output_npt_edr, 
      output_log_path=output_npt_log, 
      output_cpt_path=output_npt_cpt)

2021-06-23 07:34:49,537 [MainThread  ] [INFO ]  GROMACS Mdrun 20190 version detected
2021-06-23 07:34:49,539 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:35:50,256 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 1AKI_gppnpt.tpr -o 1AKI_npt.trr -c 1AKI_npt.gro -e 1AKI_npt.edr -g 1AKI_npt.log -cpo 1AKI_npt.cpt

2021-06-23 07:35:50,257 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:35:50,258 [MainThread  ] [INFO ]                         :-) GROMACS - gmx mdrun, 2019 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 1AKI_gppnpt.tpr -o 1AKI_npt.trr -c 1AKI_npt.gro -e 1AKI_npt.edr -g 1AKI_npt.log -cpo 1AKI_npt.cpt

Reading file 1AKI_gppnpt.tpr, VERSION 2019 (single precision)
Changing nstlist from 10 to 50, rlist from 1 to 1.115

Using 1 MPI thread
Using 8 OpenMP threads 

s

0

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [41]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = pdbCode+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop)

2021-06-23 07:35:50,290 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:35:50,311 [MainThread  ] [INFO ]  gmx energy -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_npt.edr -o 1AKI_npt_PD.xvg -xvg none < 96ccd9b2-aa0a-4a54-b2ab-09a257d517cf/instructions.in

2021-06-23 07:35:50,313 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:35:50,313 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 2 data sets
All statistics are over 51 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                    -6.0881         21    197.563   -4.64836  (bar)
Density                     1018.39        1.1    4.16549    6.74408  (kg/m^3)

2021-06-23 07:35:50,314 [MainThread  ] [INFO ]                         :-) GROMACS - gmx energy, 2019 (-:

                            GROMACS is written by

0

In [42]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein system**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms** 
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr** 
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **free MD simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 50000

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, and a total of **50,000 md steps** (100ps).

*Please note that for the sake of time this tutorial is only running 100ps of free MD, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/08_MD.html) the simulated time was 1ns (1000ps).*

In [43]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppmd_tpr = pdbCode+'_gppmd.tpr'
prop = {
    'mdp':{
        'nsteps':'50000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    },
    'simulation_type': 'free'
}

# Create and launch bb
grompp(input_gro_path=output_npt_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppmd_tpr, 
       input_cpt_path=output_npt_cpt, 
       properties=prop)

2021-06-23 07:35:50,467 [MainThread  ] [INFO ]  GROMACS Grompp 20190 version detected
2021-06-23 07:35:50,478 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_genion_top.zip
2021-06-23 07:35:50,479 [MainThread  ] [INFO ]  to:
2021-06-23 07:35:50,480 [MainThread  ] [INFO ]  ['fdc4a3b3-42d6-4037-9b1c-4c4bb9edf6fe/p2g.top', 'fdc4a3b3-42d6-4037-9b1c-4c4bb9edf6fe/posre.itp']
2021-06-23 07:35:50,481 [MainThread  ] [INFO ]  Unzipping: 
2021-06-23 07:35:50,482 [MainThread  ] [INFO ]  1AKI_genion_top.zip
2021-06-23 07:35:50,483 [MainThread  ] [INFO ]  To: 
2021-06-23 07:35:50,484 [MainThread  ] [INFO ]  fdc4a3b3-42d6-4037-9b1c-4c4bb9edf6fe/p2g.top
2021-06-23 07:35:50,487 [MainThread  ] [INFO ]  fdc4a3b3-42d6-4037-9b1c-4c4bb9edf6fe/posre.itp
2021-06-23 07:35:50,489 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:35:50,884 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f f0e7689e-0dde-486

0

<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

In [44]:
# Mdrun: Running free dynamics
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = pdbCode+'_md.trr'
output_md_gro = pdbCode+'_md.gro'
output_md_edr = pdbCode+'_md.edr'
output_md_log = pdbCode+'_md.log'
output_md_cpt = pdbCode+'_md.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppmd_tpr, 
      output_trr_path=output_md_trr, 
      output_gro_path=output_md_gro, 
      output_edr_path=output_md_edr, 
      output_log_path=output_md_log, 
      output_cpt_path=output_md_cpt)

2021-06-23 07:35:50,929 [MainThread  ] [INFO ]  GROMACS Mdrun 20190 version detected
2021-06-23 07:35:50,931 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:45:45,145 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 1AKI_gppmd.tpr -o 1AKI_md.trr -c 1AKI_md.gro -e 1AKI_md.edr -g 1AKI_md.log -cpo 1AKI_md.cpt

2021-06-23 07:45:45,147 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:45:45,148 [MainThread  ] [INFO ]                         :-) GROMACS - gmx mdrun, 2019 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 1AKI_gppmd.tpr -o 1AKI_md.trr -c 1AKI_md.gro -e 1AKI_md.edr -g 1AKI_md.log -cpo 1AKI_md.cpt

Reading file 1AKI_gppmd.tpr, VERSION 2019 (single precision)
Changing nstlist from 10 to 50, rlist from 1 to 1.116

Using 1 MPI thread
Using 8 OpenMP threads 

starting mdrun

0

<a id="mdStep3"></a>
### Step 3: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [45]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbCode+'_rms_first.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmd_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_first, 
          properties=prop)

2021-06-23 07:45:45,176 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:45:45,378 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_gppmd.tpr -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_md.trr -o 1AKI_rms_first.xvg -xvg none

2021-06-23 07:45:45,380 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:45:45,380 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2021-06-23 07:45:45,381 [MainThread  ] [INFO ]                          :-) GROMACS - gmx rms, 2019 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hi

0

In [46]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_exp, 
          properties=prop)

2021-06-23 07:45:45,401 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:45:45,601 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_gppmin.tpr -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_md.trr -o 1AKI_rms_exp.xvg -xvg none

2021-06-23 07:45:45,602 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:45:45,603 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2021-06-23 07:45:45,604 [MainThread  ] [INFO ]                          :-) GROMACS - gmx rms, 2019 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hin

0

In [47]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
}

plotly.offline.iplot(fig)


In [48]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbCode+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rgyr, 
          properties=prop)

2021-06-23 07:45:45,695 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:45:45,906 [MainThread  ] [INFO ]  echo "Backbone" | gmx gyrate -s /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_gppmin.tpr -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_md.trr -o 1AKI_rgyr.xvg -xvg none

2021-06-23 07:45:45,907 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:45:45,907 [MainThread  ] [INFO ]  Selected 4: 'Backbone'

2021-06-23 07:45:45,909 [MainThread  ] [INFO ]                         :-) GROMACS - gmx gyrate, 2019 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  A

0

In [49]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
}

plotly.offline.iplot(fig)

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str** 
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [50]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode+'_imaged_traj.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
gmx_image(input_traj_path=output_md_trr,
         input_top_path=output_gppmd_tpr,
         output_traj_path=output_imaged_traj, 
          properties=prop)

2021-06-23 07:45:45,973 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:45:46,185 [MainThread  ] [INFO ]  echo "Protein" "Protein" | gmx trjconv -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_md.trr -s /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_gppmd.tpr -fit none -o 1AKI_imaged_traj.trr -center -pbc mol -ur compact

2021-06-23 07:45:46,187 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:45:46,191 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 1: 'Protein'

2021-06-23 07:45:46,192 [MainThread  ] [INFO ]                        :-) GROMACS - gmx trjconv, 2019 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Chr

0

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [51]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = pdbCode+'_md_dry.gro'
prop = {
    'selection':  'Protein'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_md_gro,
         input_top_path=output_gppmd_tpr,
         output_str_path=output_dry_gro, 
          properties=prop)

2021-06-23 07:45:46,220 [MainThread  ] [INFO ]  Not using any container
2021-06-23 07:45:46,416 [MainThread  ] [INFO ]  echo "Protein" | gmx trjconv -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_md.gro -s /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_md_setup/biobb_wf_md_setup/notebooks/1AKI_gppmd.tpr -o 1AKI_md_dry.gro

2021-06-23 07:45:46,417 [MainThread  ] [INFO ]  Exit code 0

2021-06-23 07:45:46,418 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 1: 'Protein'

2021-06-23 07:45:46,419 [MainThread  ] [INFO ]                        :-) GROMACS - gmx trjconv, 2019 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feen

0

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [52]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

NGLWidget(max_frame=10)

<img src='trajectory.gif'></img>

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_md_gro}}: **Final structure** (snapshot) of the MD setup protocol.
 - {{output_md_trr}}: **Final trajectory** of the MD setup protocol.
 - {{output_md_cpt}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_gppmd_tpr}}: **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - {{output_genion_top_zip}}: **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliar **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
